In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

## Retrieve Data

In [ ]:
dataset = "Larger"

MAX = 12
HALF = 6
QUARTER = 4

df_allaround = pd.read_csv("F:\\VUB\\S2\\150_GIT\\50_P&D\\assignments\\parallelism\\measurements\\26-10-firefly-testing-1\\runtimes_Larger.csv", index_col=0, header=None)
df_128 = pd.read_csv("F:\\VUB\\S2\\150_GIT\\50_P&D\\assignments\\parallelism\\measurements\\firefly\\runtimes_Larger_128.csv", index_col=0, header=None)

# print(df_allaround)


In [ ]:

print(df_allaround.T)
print(df_128)


## Transform Data

In [ ]:

df_allaroundT = pd.DataFrame(df_allaround.T, columns=["SEQ","ARTICLE25_TEXT200_1", "ARTICLE_1", "TEXT_1", "ARTICLE25_TEXT200_4", "ARTICLE_4", "TEXT_4", "ARTICLE_HALF", "TEXT_HALF", "ARTICLE_MAX", "TEXT_MAX"]) # transpose the data
df_allaroundT



In [ ]:

df_128T = pd.DataFrame(df_128.T, columns=["TEXT5_MAX", "TEXT10_MAX", "TEXT15_MAX", "TEXT25_MAX", "TEXT50_MAX", "ARTICLE_50_MAX",  "ARTICLE_100_MAX", "ARTICLE_250_MAX", "ARTICLE_500_MAX", "ARTICLE_1000_MAX" ]) # transpose the data
df_128T


# Add extra columns to the df_128 dataframa

In [ ]:

df_128T["SEQ"] = df_allaroundT["SEQ"]
df_128T["ARTICLE_1"] = df_allaroundT["ARTICLE_1"]
df_128T["TEXT_1"] = df_allaroundT["TEXT_1"]

df_128T


### Removing first 5 iterations (warmup)

In [ ]:
df_allaroundT = df_allaroundT.iloc[5:]
df_128T = df_128T.iloc[5:]

df_128T


In [ ]:
print(df_128T)

print(df_128T.columns)

### Intermediary Plot

In [ ]:
data = df_allaroundT
fig = px.line(data)
fig.show()


In [ ]:

data = df_128T
fig2 = px.line(data)
fig2.show()


## Scale the columns

In [ ]:
# divide = 1000000
# divide =  1000000000 # to seconds
divide = 1000000 # to milliseconds

df_allaroundT = df_allaroundT.div(divide)

df_allaroundT


In [ ]:
# divide = 1000000
# divide =  1000000000 # to seconds
divide = 1000000 # to milliseconds

df_128T = df_128T.div(divide)

df_128T


### Intermediary Plot

In [ ]:

# scaled_columns = []
# for column in df_allaroundT.columns:
#     if "SCALED" in column:
#         scaled_columns.append(column)
# 
# data = df_allaroundT[scaled_columns]
# 
# fig = px.line(data)
# fig.show()



#### Phase 1 - Articles Parallel

In [ ]:


data = df_allaroundT[["SEQ", "ARTICLE_1",  "ARTICLE_4", "ARTICLE_HALF", "ARTICLE_MAX"]]
fig = px.line(data)

fig.show()

##### Phase 2 - Text Parallel

In [ ]:

data = df_allaroundT[["SEQ", "TEXT_1", "TEXT_4", "TEXT_HALF",  "TEXT_MAX"]]
fig = px.line(data)

fig.show()



## Extract Parameters

### AllAround Benchmarks

In [ ]:

df_allaroundT


In [ ]:

columns = [ "SEQ", "ARTICLE25_TEXT200_1", "ARTICLE_1", "TEXT_1", "ARTICLE25_TEXT200_4", "ARTICLE_4", "TEXT_4", "ARTICLE_HALF", "TEXT_HALF",  "ARTICLE_MAX", "TEXT_MAX" ]
rows = [ "mean", "median", "max", "min", "overhead", "computation_speedup", "efficiency", "application_speedup" ]

df_allaroundNew = pd.DataFrame(columns=columns, index=rows)


for column in columns:
    # print("Extracting parameters for: "+column)
    # newColumnName = column+"_SCALED"
    df_allaroundNew.loc["mean", column] = round(df_allaroundT[column].mean(),2)
    df_allaroundNew.loc["median", column] = round(df_allaroundT[column].median(),2)
    df_allaroundNew.loc["max", column] = round(df_allaroundT[column].max(),2)
    df_allaroundNew.loc["min", column] = round(df_allaroundT[column].min(),2)

# Overhead
df_allaroundNew.loc["overhead", "ARTICLE25_TEXT200_1"]  = df_allaroundNew["ARTICLE25_TEXT200_1"]["median"] / df_allaroundNew["SEQ"]["median"]
df_allaroundNew.loc["overhead", "ARTICLE_1"]  = df_allaroundNew["ARTICLE_1"]["median"] / df_allaroundNew["SEQ"]["median"]
df_allaroundNew.loc["overhead", "TEXT_1"]  = df_allaroundNew["TEXT_1"]["median"] / df_allaroundNew["SEQ"]["median"]


# TODO: Verify correctness
# Application Speedup
application_speedup = [ "ARTICLE25_TEXT200_1", "ARTICLE_1", "TEXT_1", "ARTICLE25_TEXT200_4", "ARTICLE_4", "TEXT_4", "ARTICLE_HALF", "TEXT_HALF",  "ARTICLE_MAX", "TEXT_MAX" ]

for column in application_speedup:
    # print("Application Speedup for: "+column)
    df_allaroundNew.loc["application_speedup", column]  = df_allaroundNew["SEQ"]["median"] / df_allaroundNew[column]["median"]


# TODO: Verify correctness    
# Computational Speedup
computational_speedup = [ "ARTICLE25_TEXT200_4", "ARTICLE_4", "TEXT_4", "ARTICLE_HALF", "TEXT_HALF",  "ARTICLE_MAX", "TEXT_MAX" ]

for column in computational_speedup:
    # print("Computational Speedup for: "+column)
    to = ""
    if "ARTICLE" in column and "TEXT" in column:
        to = "ARTICLE25_TEXT200_1"
    elif "ARTICLE" in column:
        to = "ARTICLE_1"
    elif "TEXT" in column:
        to = "TEXT_1"

    df_allaroundNew.loc["computation_speedup", column]  = df_allaroundNew[to]["median"] / df_allaroundNew[column]["median"]


# TODO: Verify correctness  
# Efficiency
efficiency = [ "ARTICLE25_TEXT200_4", "ARTICLE_4", "TEXT_4", "ARTICLE_HALF", "TEXT_HALF",  "ARTICLE_MAX", "TEXT_MAX" ]

for column in efficiency:
    # print("Computational Speedup for: "+column)
    to = ""
    if "ARTICLE" in column and "TEXT" in column:
        to = "ARTICLE25_TEXT200_1"
    elif "ARTICLE" in column:
        to = "ARTICLE_1"
    elif "TEXT" in column:
        to = "TEXT_1"

    workers = 0
    if "4" in column:
        workers = 4
    elif "MAX" in column:
        workers = MAX
    elif "HALF" in column:
        workers = HALF
    elif "32" in column:
        workers = QUARTER

    df_allaroundNew.loc["efficiency", column]  = df_allaroundNew[to]["median"] / (workers * df_allaroundNew[column]["median"])

df_allaroundNew


### Firefly 128

In [105]:

columns = [ "SEQ", "ARTICLE_1", "TEXT_1", "TEXT5_MAX", "TEXT10_MAX", "TEXT15_MAX", "TEXT25_MAX", "TEXT50_MAX", "ARTICLE_50_MAX",  "ARTICLE_100_MAX", "ARTICLE_250_MAX", "ARTICLE_500_MAX", "ARTICLE_1000_MAX" ]
rows = [ "mean", "median", "max", "min", "overhead", "computation_speedup", "efficiency", "application_speedup" ]

df_128New = pd.DataFrame(columns=columns, index=rows)


for column in columns:
    # print("Extracting parameters for: "+column)
    # newColumnName = column+"_SCALED"
    df_128New.loc["mean", column] = round(df_128T[column].mean(),2)
    df_128New.loc["median", column] = round(df_128T[column].median(),2)
    df_128New.loc["max", column] = round(df_128T[column].max(),2)
    df_128New.loc["min", column] = round(df_128T[column].min(),2)

# Overhead
df_128New.loc["overhead", "ARTICLE_1"]  = df_128New["ARTICLE_1"]["median"] / df_128New["SEQ"]["median"]
df_128New.loc["overhead", "TEXT_1"]  = df_128New["TEXT_1"]["median"] / df_128New["SEQ"]["median"]


# TODO: Verify correctness
# Application Speedup
application_speedup = [ "ARTICLE_1", "TEXT_1", "TEXT5_MAX", "TEXT10_MAX", "TEXT15_MAX", "TEXT25_MAX", "TEXT50_MAX", "ARTICLE_50_MAX",  "ARTICLE_100_MAX", "ARTICLE_250_MAX", "ARTICLE_500_MAX", "ARTICLE_1000_MAX" ]

for column in application_speedup:
    # print("Application Speedup for: "+column)
    df_128New.loc["application_speedup", column]  = df_128New["SEQ"]["median"] / df_128New[column]["median"]


# TODO: Verify correctness    
# Computational Speedup
computational_speedup = [ "TEXT5_MAX", "TEXT10_MAX", "TEXT15_MAX", "TEXT25_MAX", "TEXT50_MAX", "ARTICLE_50_MAX",  "ARTICLE_100_MAX", "ARTICLE_250_MAX", "ARTICLE_500_MAX", "ARTICLE_1000_MAX" ]

for column in computational_speedup:
    # print("Computational Speedup for: "+column)
    to = ""
    if "ARTICLE" in column and "TEXT" in column:
        to = "ARTICLE25_TEXT200_1"
    elif "ARTICLE" in column:
        to = "ARTICLE_1"
    elif "TEXT" in column:
        to = "TEXT_1"

    df_128New.loc["computation_speedup", column]  = df_128New[to]["median"] / df_128New[column]["median"]


# TODO: Verify correctness  
# Efficiency
efficiency = [ "TEXT5_MAX", "TEXT10_MAX", "TEXT15_MAX", "TEXT25_MAX", "TEXT50_MAX", "ARTICLE_50_MAX",  "ARTICLE_100_MAX", "ARTICLE_250_MAX", "ARTICLE_500_MAX", "ARTICLE_1000_MAX" ]

for column in efficiency:
    # print("Computational Speedup for: "+column)
    to = ""
    if "ARTICLE" in column and "TEXT" in column:
        to = "ARTICLE25_TEXT200_1"
    elif "ARTICLE" in column:
        to = "ARTICLE_1"
    elif "TEXT" in column:
        to = "TEXT_1"

    workers = 0
    if "4" in column:
        workers = 4
    elif "MAX" in column:
        workers = MAX
    elif "HALF" in column:
        workers = HALF
    elif "32" in column:
        workers = QUARTER

    df_128New.loc["efficiency", column]  = df_128New[to]["median"] / (workers * df_128New[column]["median"])


df_128New


,SEQ,ARTICLE_1,TEXT_1,TEXT5_MAX,TEXT10_MAX,TEXT15_MAX,TEXT25_MAX,TEXT50_MAX,ARTICLE_50_MAX,ARTICLE_100_MAX,ARTICLE_250_MAX,ARTICLE_500_MAX,ARTICLE_1000_MAX
mean,7058.89,9856.84,44761.67,3242.93,2303.23,1978.97,1790.88,1593.45,1413.92,1383.48,1413.0,1373.47,1384.67
median,7057.69,9833.57,43856.01,3228.84,2266.91,1956.5,1793.46,1585.06,1408.94,1375.59,1384.71,1365.02,1379.09
max,7112.03,9992.47,47313.27,3433.74,2466.14,2139.41,1837.3,1648.04,1541.26,1440.65,1590.09,1460.54,1417.88
min,7025.66,9794.03,43563.89,3144.96,2222.35,1940.25,1741.1,1546.79,1353.82,1347.8,1346.06,1345.94,1356.13
overhead,NaN,1.393313,6.213933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
computation_speedup,NaN,NaN,NaN,13.58259,19.346163,22.415543,24.453297,27.66836,6.97941,7.14862,7.101538,7.203975,7.130477
efficiency,NaN,NaN,NaN,1.131883,1.61218,1.867962,2.037775,2.305697,0.581618,0.595718,0.591795,0.600331,0.594206
application_speedup,NaN,0.717714,0.160929,2.185828,3.113353,3.607304,3.935237,4.452633,5.00922,5.130664,5.096872,5.170393,5.117643


## Plot Data

#### Median

In [ ]:


data = df_allaroundNew.loc["median", 'SEQ':]
fig = px.line(y=data, x=df_allaroundNew.columns)
fig.show()



In [ ]:

# Text sequential
# 10 or 20 or 50
# 100
# 200 (current)

# Articles sequential
# ...
# 50 (this)


# data = df_allaroundNew.loc["median", 'SEQ':'ARTICLE_TEXT_MAX']
# 
# fig = px.line(data)
# 
# fig.show()

#### Computation Speedup

In [ ]:
data = df_allaroundNew.loc["computation_speedup", 'ARTICLE25_TEXT200_4':]

fig = px.line(data)

fig.show()

#### Efficiency

In [ ]:
data = df_allaroundNew.loc["efficiency", 'ARTICLE25_TEXT200_4':]

fig = px.line(data)

fig.show()

#### Application Speedup

In [ ]:
data = df_allaroundNew.loc["application_speedup", 'ARTICLE25_TEXT200_1':]

fig = px.line(data)

fig.show()

### Firefly 128

#### Computation Speedup


In [111]:

data = df_128New.loc["computation_speedup", 'TEXT5_MAX':'TEXT50_MAX']

fig = px.line(y=data, x=[5, 10, 15, 25, 50])

fig.show()


In [112]:
data = df_128New.loc["computation_speedup", 'ARTICLE_50_MAX':'ARTICLE_1000_MAX']

fig = px.line(y=data, x=[50, 100, 250, 500, 1000])

fig.show()

#### Efficiency

In [108]:
data = df_128New.loc["efficiency", 'TEXT5_MAX': 'TEXT50_MAX']

fig = px.line(y=data, x=[5, 10, 15, 25, 50])

fig.show()

In [113]:

data = df_128New.loc["efficiency", 'ARTICLE_50_MAX':'ARTICLE_1000_MAX']

fig = px.line(y=data, x=[50, 100, 250, 500, 1000])

fig.show()


#### Application Speedup

In [115]:
data = df_128New.loc["application_speedup", 'TEXT5_MAX': 'TEXT50_MAX']

fig = px.line(y=data, x=[5, 10, 15, 25, 50])

fig.show()

In [116]:

data = df_128New.loc["application_speedup", 'ARTICLE_50_MAX':'ARTICLE_1000_MAX']

fig = px.line(y=data, x=[50, 100, 250, 500, 1000])

fig.show()